In [1]:
from query.query_engine import create_query_engine
from query.routing import create_routing_engine
from evaluation.eval_with_llamaindex import evaluate, evaluate_multiple_experiments

from loguru import logger
from pprint import pprint
import pandas as pd
import nest_asyncio
nest_asyncio.apply()
import sys
import os

#set level of logging
logger.remove()
logger.add(sys.stderr, level="INFO")
os.chdir("..")

# BASIC RAG

In [ ]:
query_engine = create_query_engine()

response = query_engine.query("What are the conditions required for a marriage to be considered valid in France?")

print(response)

# Embeddings evaluation

In [ ]:
query_engine_fastembed = create_query_engine(embedding_model="intfloat/multilingual-e5-large")
query_engine_mistral = create_query_engine(embedding_model="mistral-embed")
query_engine_ada = create_query_engine(embedding_model="text-embedding-ada-002")

In [ ]:
exp_to_query_engine = {
    "mistral_embed": query_engine_mistral,
    "fastembed": query_engine_fastembed,
    "ada": query_engine_ada,
}

scores_df, deeps_df = evaluate_multiple_experiments(
    experiment_to_query_engine=exp_to_query_engine,
    general_exp_name="embeddings",
    list_metrics=["context_relevancy"],
    code_name="code_civil"
)


#  Nodes window

## 1. Evaluate context relevancy

In [ ]:
query_engine_window_1 = create_query_engine(use_window_nodes=True, nodes_window_size=1, recreate_collection=True)
query_engine_window_3 = create_query_engine(use_window_nodes=True, nodes_window_size=3, recreate_collection=True)

In [ ]:
from llama_index.core.schema import MetadataMode

experiment_to_query_engine = {
    "window_1": query_engine_window_1,
    "window_3": query_engine_window_3,
}
mean_scores_df, deep_dfs = evaluate_multiple_experiments(
    experiment_to_query_engine=experiment_to_query_engine,
    list_metrics=["context_relevancy"],
    general_exp_name="window",
    code_name="code_civil",
    metadata_mode=MetadataMode.NONE,
)


## 2. evaluate faithfulness

In [ ]:
query_engine_base = create_query_engine()
query_engine_window_3 = create_query_engine(use_window_nodes=True, nodes_window_size=3)

In [ ]:
experiment_to_query_engine = {
    "base": query_engine_base,
    "window_3": query_engine_window_3,
}
mean_scores_df, deep_dfs = evaluate_multiple_experiments(
    experiment_to_query_engine=experiment_to_query_engine,
    list_metrics=["faithfulness"],
    general_exp_name="window_faithfulness",
    code_name="code_civil",
)

# Hybrid search

In [ ]:
query_engine_hybrid_search_alpha_0_5 = create_query_engine(hybrid_search=True, sparse_top_k=5)
query_engine_hybrid_search_alpha_0_2 = create_query_engine(hybrid_search=True, sparse_top_k=5, hybrid_search_alpha=0.2)
query_engine_hybrid_search_alpha_0_8 = create_query_engine(hybrid_search=True, sparse_top_k=5, hybrid_search_alpha=0.8)

In [ ]:
experiment_to_query_engine = {
    "alpha_0_5": query_engine_hybrid_search_alpha_0_5,
    "alpha_0_2": query_engine_hybrid_search_alpha_0_2,
    "alpha_0_8": query_engine_hybrid_search_alpha_0_8,
}


mean_scores_df, deep_dfs = evaluate_multiple_experiments(
    experiment_to_query_engine=experiment_to_query_engine,
    list_metrics=["faithfulness", "context_relevancy"],
    general_exp_name="hybrid_search",
    code_name="code_civil",
)

# Query rewriting

In [ ]:
query_engine_base = create_query_engine(hybrid_search=True, 
                                     sparse_top_k=5, 
                                     hybrid_search_alpha=0.8)

query_engine_rewriter = create_query_engine(query_rewrite=True, 
                                     hybrid_search=True, 
                                     sparse_top_k=5, 
                                     hybrid_search_alpha=0.8)

In [ ]:
experiment_to_query_engine = {
    "base": query_engine_base,
    "rewriting_query": query_engine_rewriter,
}
mean_scores_df, deep_dfs = evaluate_multiple_experiments(
    experiment_to_query_engine=experiment_to_query_engine,
    list_metrics=["faithfulness", "answer_relevancy"],
    general_exp_name="query_rewriting",
    code_name="code_civil",
)

# Routing

In [ ]:
routing_query_engine = create_routing_engine()

In [4]:
question = "Quelles sont les conditions pour bénéficier d'un congé sabbatique \n Comment sont imposées les plus-values immobilières"
res = routing_query_engine.query(question)
res